# Streaming Least Squares (SLS) for Time Series Anomaly Detection
<img src="http://fraudcrimeprevention.files.wordpress.com/2012/06/shutterstock_664253742.jpg" width="200"/>

## Algorithm explained
1. Generate sliding windows
2. For each window, run ordinary least squares (aka. linear regression) and compute the regression residual as the anomaly of that window
3. These windows are sorted by anomaly score with overlaping windows of lower anomaly score dropped from the results
4. The results are classified into levels. Levels are formed on the basis that anomalies in the same level have similar scores. This process of finding level groups is similar to doing K-Means in one dimension.
5. The clustering in step 4 also give us the automatic thresholds to determine the level of each anomaly.

### Efficient streaming least squares
Ordinary Least Squares can be solved by using the normal equation, i.e. $$\beta = (X^TX)^{-1}X^Ty$$

The residual is of the the linear reregression is $||r||$, in which $$r = X\beta - y$$

Substituting $\beta$ from the normal equation gives us

$$
\begin{array}{rcl}
r& = & X (X^TX)^{-1}X^Ty - y\\
 & = & \big(X (X^TX)^{-1}X^T - I\big) y.
\end{array}
$$

Note that for a fixed window size, the matrix $X$ is also fixed and has the following form
$$
\begin{array}{cc}
0 & 1 \\
1 & 1 \\
2 & 1 \\
... & .. \\
w-1 & 1
\end{array}
$$

The matrix $X (X^TX)^{-1}X^T - I$, of size $w\times w$, can thus be pre-computed. Then, the regression residual for each window can be computed efficiently by a matrix multiply followed by a norm.

In the training phase, we can stack the sliding windows horizontally as a matrix and compute the residuals of all windows by one $(w\times w) \times (w\times n)$ matrix multiplication, in which $n$ is the number of sliding windows.

## Usage
SLS algorithm has 3 main parameters
- **Time-to-detection** (aka. **window size**): this is the period of time that it takes a user to confirm that the data is abnormal. Example: 20 minutes. <u>Default value</u>: 20
- **Number of levels**: specifies the number of severity levels. The higher this number, the more granular the anomalies classification will be. A number too high (e.g. 30) may lead to unncessary noise, i.e. the algorithm may classify benign patterns as anomalies, albeit with low severity level. <u>Default value</u>: 5
- **Max number of anomalies**: this parameter is optional. If you have rough idea of how many anomalies in the historical data, you can provide that as weak supervision. <u>Default value</u>: `None`

## Advantages of SLS over other algorithms
#### Accurate
SLS has a training phase and hence can detect the pattern over the entire period.

Holt-Winters and other streaming-only algorithms classify based on past recent values. This leads to:
- many false positives after an incident is already mitigated
- longer time-to-detection because it has a long history dependency

#### Automatic Thresholding
#### Severity Levels
SLS can output the severity level of an anomaly.
#### Stateless 
SLS is stateless. The model isn't changed during when operationalized and hence is easier to debug, i.e. one doesn't need to rerun the algorithm from the beginning of time to debug.